In [72]:
import re
import numpy as np
import pandas as pd
import feather
import xgboost as xgb

from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OutputCodeClassifier
  
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
  
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV
from sklearn import svm
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import MultinomialNB

from sklearn.utils.testing import assert_array_equal
from sklearn.utils.testing import assert_equal
from sklearn.utils.testing import assert_almost_equal
from sklearn.utils.testing import assert_true
from sklearn.utils.testing import assert_false
from sklearn.utils.testing import assert_raises
  
from sklearn.utils.testing import assert_greater
import pickle

from sklearn.metrics import log_loss, confusion_matrix, f1_score, accuracy_score
from sklearn.calibration import CalibratedClassifierCV

from utils import multi_log_loss

In [2]:

def save_classifier(fname, clf):
    # save the classifier
    with open(fname, 'wb') as fid:
        pickle.dump(clf, fid)

def load_classifier(fname):
    # load it again
    with open(fname, 'rb') as fid:
        clf = pickle.load(fid)
        return clf

In [3]:
df_train = np.load('../cache/train_p_stage1_fe2.npy')

In [4]:
df_test = np.load('../cache/test_p_stage1_fe2.npy')

In [5]:
df = pd.read_csv('../cache/stage1_labels.csv')

In [6]:
df.head()

,y
0,1
1,2
2,2
3,3
4,4


In [7]:
y = df['y'].values

In [8]:
df_train.shape

(3321, 10233)

In [9]:
df1 = pd.read_csv('../cache/stage1_p_test_labels.csv')
test_labels = df1['y'].values

In [10]:
wts_per_class = np.load('../cache/stage1_train_weights_per_class.npy')
wts_per_class = wts_per_class.tolist()
print(wts_per_class)

{1: 4.846830985915493, 2: 6.347345132743363, 3: 36.31460674157304, 4: 3.8411078717201166, 5: 12.723140495867769, 6: 11.076363636363636, 7: 2.484784889821616, 8: 173.78947368421052, 9: 88.75675675675676}


In [93]:
pd.value_counts(y)

7    953
4    686
1    568
2    452
6    275
5    242
3     89
9     37
8     19
dtype: int64

In [94]:
2 * (953/3321) * 0.11/ ((953/3321) + 0.11)

0.15903694882083882

In [95]:
pd.value_counts(test_labels)

7    101
1     94
4     65
2     46
5     25
6     22
3      7
9      6
8      2
dtype: int64

In [96]:
df_test.shape

(368, 10233)

In [97]:
2 * (101/368) * 0.11/ ((101/368) + 0.11)

0.15705400056545096

In [11]:
# def test_ovo_gridsearch():
ovo = OneVsOneClassifier(svm.SVC(probability=True))
Cs = [0.01, 0.02, 0.05, 0.09]
cv = GridSearchCV(ovo, {'estimator__C': Cs})
cv.fit(df_train, y)
best_C = cv.best_estimator_.estimators_[0].C
assert_true(best_C in Cs)


In [12]:
print(best_C)

0.01


In [73]:
ovo = CalibratedClassifierCV(OneVsOneClassifier(svm.SVC(probability=True, C=0.01)))

In [75]:
ovo.fit(df_train, y)

CalibratedClassifierCV(base_estimator=OneVsOneClassifier(estimator=SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          n_jobs=1),
            cv=3, method='sigmoid')

In [76]:
test_p = ovo.predict_proba(df_test)

In [83]:
test_preds = test_p

In [84]:
score2 = log_loss(test_labels, test_preds, labels = range(1,10))
print('stage1 partial multi_log_loss: {}'.format(score2))

fscore = f1_score(test_labels, test_preds.argmax(axis=1)+1, labels = list(range(1,10)), average='micro')
print('stage1 partial f1_score: {}'.format(fscore))

acc = accuracy_score(test_labels, test_preds.argmax(axis=1)+1)
print('stage1 partial accuracy: {}'.format(acc))

print(confusion_matrix(test_labels, test_preds.argmax(axis=1)+1, labels = list(range(1,10))))

stage1 partial multi_log_loss: 1.8186399268986517
stage1 partial f1_score: 0.27445652173913043
stage1 partial accuracy: 0.27445652173913043
[[  0   0   0   0   0   0  94   0   0]
 [  0   0   0   0   0   0  46   0   0]
 [  0   0   0   0   0   0   7   0   0]
 [  0   0   0   0   0   0  65   0   0]
 [  0   0   0   0   0   0  25   0   0]
 [  0   0   0   0   0   0  22   0   0]
 [  0   0   0   0   0   0 101   0   0]
 [  0   0   0   0   0   0   2   0   0]
 [  0   0   0   0   0   0   6   0   0]]


In [85]:
np.unique(df['y'].values)

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [86]:
clf = OneVsRestClassifier(CalibratedClassifierCV(LinearSVC())).fit(df_train, df['y'].values)

In [87]:
test_preds = clf.predict_proba(df_test)

In [88]:
print(test_preds)

[[ 0.19963798  0.12434148  0.02703923 ...,  0.27170648  0.00627771
   0.0106242 ]
 [ 0.19012832  0.1268235   0.04303355 ...,  0.22421147  0.00585256
   0.01101572]
 [ 0.14800983  0.14545672  0.03859867 ...,  0.27126375  0.00573652
   0.0078683 ]
 ..., 
 [ 0.16209228  0.13924399  0.0353278  ...,  0.27202533  0.00589397
   0.00867665]
 [ 0.14869046  0.14712454  0.0173737  ...,  0.33439177  0.00563372
   0.00930825]
 [ 0.17616286  0.13069378  0.03403332 ...,  0.2665317   0.0061262
   0.00940196]]


In [92]:
score2 = log_loss(test_labels, test_preds, labels=[1,2,3,4,5,6,7,8,9])
print('stage1 partial multi_log_loss: {}'.format(score2))

fscore = f1_score(test_labels, test_preds.argmax(axis=1)+1, labels = list(range(1,10)), average='micro')
print('stage1 partial f1_score: {}'.format(fscore))

acc = accuracy_score(test_labels, test_preds.argmax(axis=1)+1)
print('stage1 partial accuracy: {}'.format(acc))

print(confusion_matrix(test_labels, test_preds.argmax(axis=1)+1, labels = list(range(1,10))))

stage1 partial multi_log_loss: 1.7874586410646618
stage1 partial f1_score: 0.28804347826086957
stage1 partial accuracy: 0.28804347826086957
[[ 0  0  0  5  0  0 89  0  0]
 [ 0  0  0  1  0  0 45  0  0]
 [ 0  0  0  1  0  0  6  0  0]
 [ 0  0  0  7  0  0 58  0  0]
 [ 0  0  0  2  0  0 23  0  0]
 [ 0  0  0  2  0  0 20  0  0]
 [ 0  0  0  2  0  0 99  0  0]
 [ 0  0  0  0  0  0  2  0  0]
 [ 0  0  0  1  0  0  5  0  0]]


In [38]:
Y = df['y'].values

In [54]:
ovo = MultinomialNB()
Y_pred =ovo.fit(abs(df_train), Y).predict_proba(df_test)

In [57]:
Y_pred = Y_pred.clip(min=0.05, max=0.95)

In [59]:
score2 = log_loss(test_labels, Y_pred, labels=range(1,10))
print('stage1 partial multi_log_loss: {}'.format(score2))

fscore = f1_score(test_labels, Y_pred.argmax(axis=1)+1, labels = list(range(1,10)), average='micro')
print('stage1 partial f1_score: {}'.format(fscore))

acc = accuracy_score(test_labels, Y_pred.argmax(axis=1)+1)
print('stage1 partial accuracy: {}'.format(acc))

print(confusion_matrix(test_labels, Y_pred.argmax(axis=1)+1, labels = list(range(1,10))))

stage1 partial multi_log_loss: 2.9597867651212026
stage1 partial f1_score: 0.11413043478260869
stage1 partial accuracy: 0.11413043478260869
[[ 9  0 23  6  0 19  3  9 25]
 [ 1  4 15  0  2 11  5  0  8]
 [ 2  0  4  0  0  0  0  0  1]
 [ 3  1 26  5  0  4  2  2 22]
 [ 2  0 10  0  2  8  2  0  1]
 [ 1  0  7  0  0  5  1  1  7]
 [ 1  6 19  1  5 14 10  6 39]
 [ 0  0  0  0  0  0  0  0  2]
 [ 0  0  3  0  0  0  0  0  3]]
